In [36]:
from src.motion_control.LocalNavigator import LocalNavigator
from src.motion_control.ThymioController import ThymioController
from src.motion_control.motion_control_final import MotionControl
from src.vision.vision import ComputerVision
from src.global_navigation.global_navigation import AStarNavigation
import time
import numpy as np
from src.KalmanFilter import KalmanFilter
from src.Image_EKF import Image_EKF
from src.PositionUpdate import PositionUpdate
import cv2
from src.Constants import MAIN_LOOP_DELAY

vision = ComputerVision()
image_var = vision.get_map()

global_navigation = AStarNavigation(safety=5, file_path=None)
global_path = global_navigation.new_path_with_var(image_var)

#print(global_path)
#Motion control, filtering, local navigation
thymio = ThymioController()
thymio.connect(timeout=5)
local_navigator = LocalNavigator(thymio)
motion_control = MotionControl(thymio, global_path, global_navigation)




Searching for Thymio...
Thymio connected!


In [5]:
%load_ext autoreload
%autoreload 2


In [6]:
%reload_ext autoreload    

In [11]:
STATUS_IN_MOVE = 1
STATUS_OBSTACLE_AVOIDANCE = 2    
STATUS_KIDNAPPING = 3


camera, aruco_dict, aruco_params, newcameramtx, dist, mtx =Image_EKF.initialize_camera_and_calibration()
position_update = PositionUpdate(thymio)


In [38]:
position_update = PositionUpdate(thymio)

In [39]:
thymio.set_speed(20,20)

In [40]:
try:
    # Define path (list of waypoints)
    while True:
        ret, img = camera.read()
        
        #filtering
        position_update.path_predicition(newcameramtx, dist, aruco_dict, aruco_params, mtx, img)
        cv2.putText(img, "yaw aruco"+ str(np.round(np.degrees((position_update.kf.Z[2])),3)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(img, "yaw"+str(np.round(np.degrees(position_update.kf.theta))), (400, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        PositionUpdate.draw_path(img,position_update.odom_centers,(255,255,0),10)
        PositionUpdate.draw_path(img, position_update.marker_centers,(255,0,0),5)
    
        cv2.imshow("ArUco Detection", img)
      

        if cv2.waitKey(1) & 0xFF == ord('q'):
            thymio.stop()
            break


        if local_navigator.kidnapping_detect():
            thymio.stop()
            local_navigator.put_back_detecting()
            current_pos_x, current_pos_y = global_navigation.pixel_to_grid(position_update.marker_centers[-1][0], position_update.marker_centers[-1][1])
            new_path = global_navigation.new_path(current_pos_x, current_pos_y)
            motion_control.setup_new_path(new_path, position_update.kf.theta)
            
            #TODO: 1.get current position from filtering 2.Compute new path
        #     
        # if local_navigator.obstacle_detected():
        #     thymio.stop()
        #     local_navigator.handle_obstacle()
        #     #TODO: 1Compute new path
        # 
        # 
        target_position = motion_control.path[motion_control.path_iterator]
        target_orientation = motion_control.calculate_target_orientation(target_position)
        motion_control.align_to_target_orientation(target_orientation)
        motion_control.move_to_target(target_position, [position_update.marker_centers[-1][0], position_update.marker_centers[-1][1], position_update.kf.theta])
        
       
        
        
        time.sleep(MAIN_LOOP_DELAY)

finally:
    thymio.disconnect()
    cv2.destroyAllWindows()

[[-0.45 -0.45  0.  ]
 [ 0.45 -0.45  0.  ]
 [ 0.45  0.45  0.  ]
 [-0.45  0.45  0.  ]] [[0.]
 [0.]
 [0.]] [[-14.04119   ]
 [ -7.55614   ]
 [ 30.48947586]] [[1.45636124e+03 0.00000000e+00 9.73013488e+02]
 [0.00000000e+00 1.46761352e+03 5.65119529e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[ 1.38676388e-01 -8.93281614e-01 -1.56126800e-03  1.21200124e-03
   1.30504271e+00]]
[[-0.45 -0.45  0.  ]
 [ 0.45 -0.45  0.  ]
 [ 0.45  0.45  0.  ]
 [-0.45  0.45  0.  ]] [[0.]
 [0.]
 [0.]] [[-14.00009   ]
 [ -7.55719   ]
 [ 30.48947586]] [[1.45636124e+03 0.00000000e+00 9.73013488e+02]
 [0.00000000e+00 1.46761352e+03 5.65119529e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[ 1.38676388e-01 -8.93281614e-01 -1.56126800e-03  1.21200124e-03
   1.30504271e+00]]
[[-0.45 -0.45  0.  ]
 [ 0.45 -0.45  0.  ]
 [ 0.45  0.45  0.  ]
 [-0.45  0.45  0.  ]] [[0.]
 [0.]
 [0.]] [[-14.00355   ]
 [ -7.5571    ]
 [ 30.48947586]] [[1.45636124e+03 0.00000000e+00 9.73013488e+02]
 [0.00000000e+00 1.46761352e+03